# 实验2 电阻的温度特性
> Copyright(c) 2022 GeorgeDong32. All Rights Reserved.

## 测量铜电阻、NTC、PTC的温度特性

In [ ]:
from cProfile import label
import scipy.constants as cst 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# plt防止中文乱码
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
#拟合函数定义
def RCu(t,a,R0):
    return a*t*R0 + R0

def lnRNTC(T,A,B):
    return B/T + np.log(A)

def lnRPTC(T,A,B):
    return np.log(A) + B * T

In [ ]:
#读取数据
dCu = pd.read_csv("Cu.csv")
dNTC = pd.read_csv("NTC.csv")
dPTC = pd.read_csv("PTC.csv")

tCu = dCu.iloc[:,0]
rCu = (dCu.iloc[:,1] + dCu.iloc[:,2]) / 2

TNTC = dNTC.iloc[:,0] + 273
rNTC = (dNTC.iloc[:,1] + dNTC.iloc[:,2]) / 2
lnrNTC = np.log(rNTC)

TPTC = (dPTC.iloc[:,0] + 244)
rPTC = (dPTC.iloc[:,1] + dPTC.iloc[:,2]) / 2
lnrPTC = np.log(rPTC)

popt1,pcov1=curve_fit(RCu,tCu,rCu)
rCu_fit = RCu(tCu,popt1[0],popt1[1])

popt2,pcov2=curve_fit(lnRNTC,TNTC,lnrNTC)
lnrNTC_fit = lnRNTC(TNTC,popt2[0],popt2[1])

popt3,pcov3=curve_fit(lnRPTC,TPTC,lnrPTC)
lnrPTC_fit = lnRPTC(TPTC,popt3[0],popt3[1])

### 铜电阻图像

In [ ]:
plt.figure(dpi=150)
plt.scatter(tCu,rCu,label="铜电阻测量值")
plt.plot(tCu,rCu_fit,label="铜电阻拟合曲线")
plt.title("铜电阻温度特性曲线")
plt.xlabel("t/°C")
plt.ylabel("$R_{Cu}/\Omega$")
plt.annotate("$R_{Cu}$ = %.2f(1 + %.6ft)" %(popt1[1],popt1[0]), xy=(30,57))
plt.legend()
print(rCu)

### NTC电阻图像

In [ ]:
plt.figure(dpi=150)
plt.scatter(TNTC,lnrNTC,label="NTC电阻测量值")
plt.plot(TNTC,lnrNTC_fit,label="NTC电阻拟合曲线")
plt.title("NTC电阻温度特性曲线")
plt.xlabel("T/K")
plt.ylabel("$lnR_{NTC}/ln \Omega$")
plt.annotate("$lnR_{NTC}$ = %.3f/T + %.3f" %(popt2[1],np.log(popt2[0])), xy=(303,6))
plt.legend()
print(popt2[0])
print(rNTC)

### PTC电阻图像

In [ ]:
plt.figure(dpi=150)
plt.scatter(TPTC+29,lnrPTC,label="PTC电阻测量值")
plt.plot(TPTC+29,lnrPTC_fit,label="PTC电阻拟合曲线")
plt.title("PTC电阻温度特性曲线")
plt.xlabel("T/K")
plt.ylabel("$lnR_{PTC}/ln\Omega$")
plt.annotate("$lnR_{PTC}$ = ln(%.3f) + %.4fT" %(popt3[0],popt3[1]), xy=(300,7.25))
plt.legend()
print(rPTC)

### 铜电阻数据计算（逐差法）

In [ ]:
i = 7
sum = 0
while (i > 0):
    sum += rCu[i+7] - rCu[i]
    i = i - 1
aCu = sum / (7 * 35) 
print(aCu/55)
print(TNTC)

### 补充图像

#### $R_{NTC} - T$

In [ ]:
plt.figure(dpi=150)
plt.scatter(TNTC,rNTC,label="NTC电阻测量值")
plt.plot(TNTC,np.exp(lnrNTC_fit),label="NTC电阻拟合曲线")
plt.title("NTC电阻温度特性曲线")
plt.xlabel("T/K")
plt.ylabel("$R_{NTC}/ \Omega$")
#plt.annotate("$lnR_{NTC}$ = %.3f/T + %.3f" %(popt2[1],np.log(popt2[0])), xy=(303,6))
plt.savefig(".\Pics\ NTC电阻温度特性曲线.png")

#### $lnR_{NTC}-\frac{1}{T}$

In [ ]:
plt.figure(dpi=150)
plt.scatter(1/TNTC,lnrNTC,label="NTC电阻测量值")
plt.plot(1/TNTC,lnrNTC_fit,label="NTC电阻拟合曲线")
plt.title("$lnR_{NTC}-\\frac{1}{T}$曲线")
plt.xlabel("$T^{-1}/K^{-1}$")
plt.ylabel("$lnR_{NTC}$")
#plt.annotate("$lnR_{NTC}$ = %.3f/T + %.3f" %(popt2[1],np.log(popt2[0])), xy=(303,6))
plt.legend()
plt.savefig(".\Pics\lnRNTC-T.png")
print(-3381.21/TNTC**2)

#### $R_{PTC} - T$

In [ ]:
from matplotlib.pyplot import savefig


plt.figure(dpi=150)
plt.scatter(TPTC+29,rPTC,label="PTC电阻测量值")
plt.plot(TPTC+29,np.exp(lnrPTC_fit),label="PTC电阻拟合曲线")
plt.title("PTC电阻温度特性曲线")
plt.xlabel("T/K")
plt.ylabel("$R_{PTC}/ \Omega$")
plt.annotate("$lnR_{PTC}$ = ln(%.3f) + %.4fT" %(popt3[0],popt3[1]), xy=(300,7.25))
plt.legend()
plt.savefig(".\Pics\PTC电阻温度特性曲线.png")

#### $ln(R_{PTC})-T$

In [ ]:
plt.figure(dpi=150)
plt.scatter(TPTC+29,lnrPTC,label="PTC电阻测量值")
plt.plot(TPTC+29,lnrPTC_fit,label="PTC电阻拟合曲线")
plt.title("$ln(R_{PTC})-T$曲线")
plt.xlabel("T/K")
plt.ylabel("$lnR_{PTC}$")
plt.annotate("$lnR_{PTC}$ = ln(%.3f) + %.4fT" %(popt3[0],popt3[1]), xy=(300,7.25))
plt.legend()
plt.savefig(".\Pics\lnRPTC-T.png")